# Get reception metadata

Pairs our matched books with reception metadata.

In [8]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

In [2]:
match = pd.read_csv('allmatchedbooks.tsv', sep = '\t')

In [4]:
match.loc[match.docid.duplicated(keep = False), :]

,docid,tm_author,tm_title,tm_year,review_year,rownum,review_title,prize_author,bestseller_year,is_bestseller,obscure,firstpub,birthyear,us_national,lowest_date,standard_auth
506,uc1.$b348168,"allen, hervey",anthony adverse,1933.0,1933.0,838.0,anthony adverse.,False,1933.0,True,False,1933,1889.0,False,1933,"allen, hervey"
507,uc1.$b348168,"allen, hervey",anthony adverse,1933.0,1933.0,838.0,anthony adverse.,False,1934.0,True,False,1933,1889.0,False,1933,"allen, hervey"
558,mdp.39015005776847,"mitchell, margaret",gone with the wind,1936.0,1936.0,3218.0,gone with the wind.,True,1936.0,True,False,1936,1900.0,True,1936,"mitchell, margaret"
559,mdp.39015005776847,"mitchell, margaret",gone with the wind,1936.0,1936.0,3218.0,gone with the wind.,True,1937.0,True,False,1936,1900.0,True,1936,"mitchell, margaret"
1152,mdp.39015066076368,"hemingway, ernest",for whom the bell tolls,1940.0,1940.0,4292.0,for whom the bell tolls.,False,1940.0,True,False,1940,1899.0,True,1940,"hemingway, ernest"
1153,mdp.39015066076368,"hemingway, ernest",for whom the bell tolls,1940.0,1940.0,4292.0,for whom the bell tolls.,False,1941.0,True,False,1940,1899.0,True,1940,"hemingway, ernest"
2434,mdp.39015004063601,"steinbeck, john",the grapes of wrath,1939.0,1939.0,2077.0,the grapes of wrath.,True,1939.0,True,False,1939,1902.0,True,1939,"steinbeck, john"
2435,mdp.39015004063601,"steinbeck, john",the grapes of wrath,1939.0,1939.0,2077.0,the grapes of wrath.,True,1940.0,True,False,1939,1902.0,True,1939,"steinbeck, john"
2618,uc1.b3687674,"edmonds, walter dumaux",drums along the mohawk,1936.0,1936.0,1649.0,drums along the mohawk.,False,1936.0,True,False,1936,1903.0,True,1936,"edmonds, walter duma"
2619,uc1.b3687674,"edmonds, walter dumaux",drums along the mohawk,1936.0,1936.0,1649.0,drums along the mohawk.,False,1937.0,True,False,1936,1903.0,True,1936,"edmonds, walter duma"


In [5]:
tokeep = []
already = set()
for idx in match.docid:
    if idx in already:
        tokeep.append(False)
    else:
        tokeep.append(True)
    already.add(idx)

In [6]:
print(match.shape)
match = match.loc[tokeep, :]
match.shape

(5519, 16)


(5497, 16)

In [14]:
def get_path(year):
    root = '/Users/tunder/Dropbox/python/avant/'
    if year < 1931:
        path = root + '1916-30revs/fiction' + str(year - 1904) + '_' + str(year) + '.tsv'
    elif year < 1941:
        path = root + '1930srevs/nonconsumptive' + str(year - 1904) + '_' + str(year) + '.tsv'
    else:
        path = root + '1940srevs/nonconsumptive' + str(year - 1904) + '_' + str(year) + '.tsv'
        
    return path

In [17]:
def get_wordcount(citation, themedian):
    if pd.isnull(citation):
        return themedian
    token = citation.split()[-1].lower()
    if token.endswith('w'):
        token = token.replace('o', '0')
        token = token.replace('l', '1')
        token = token.replace('w', '')
        token = token.replace('s', '5')
        token = token.replace('i', '1')
        token = token.replace('b', '8')
        try:
            count = int(token)
        except:
            count = themedian
        return count
    else:
        return themedian
        

In [23]:
docdict = dict()


for year, df in match.groupby('review_year'):
    try:
        year = int(year)
    except:
        print("ERROR:", year)
        continue
    
    path = get_path(year)
    reviews = pd.read_csv(path, sep = '\t', low_memory = False)
    allcounts = []
    for cite in reviews.citation:
        wordcount = get_wordcount(cite, 0)
        if wordcount > 0:
            allcounts.append(wordcount)
    themedian = np.median(allcounts)
    print(year, themedian)
    
    for idx, row in df.iterrows():
        rownum = int(row['rownum'])
        docid = row['docid']
        reviewrow = reviews.iloc[rownum, :]
        revtitle = reviewrow['booktitle'].lower()
        title = row['review_title']
        ratio = fuzz.ratio(title, revtitle)
        if ratio < 80:
            print(title, revtitle)
        thebookindex = reviewrow['bookindex']
        allreviews = reviews.loc[reviews.bookindex == thebookindex, :]
        if year < 1931:
            sentiment = reviewrow['avgsentiment']
        else:
            sentiment = reviewrow['avgsentwmissing']
        wordcount = 0
        for cite in allreviews.citation:
            words = get_wordcount(cite, themedian)
            if words < 5000:
                wordcount += words
            else:
                wordcount += themedian
        docdict[docid] = (sentiment, wordcount)      
    
    

1916 280.0
1917 270.0
1918 300.0
1919 250.0
1920 300.0
1921 300.0
1922 350.0
1923 450.0
1924 360.0
1925 300.0
1926 320.0
1927 280.0
1928 250.0
1929 250.0
1930 250.0
1931 280.0
1932 280.0
1933 250.0
1934 280.0
1935 330.0
1936 320.0
1937 310.0
1938 300.0
1939 250.0
1940 240.0
1941 250.0
1942 240.0
1943 270.0
1944 210.0
1945 230.0
1946 240.0
1947 180.0
1948 180.0
1949 200.0
1950 220.0


In [24]:
docdict['mdp.39015005776847']

(3.722122498480001, 21900.0)

In [25]:
match.head()

,docid,tm_author,tm_title,tm_year,review_year,rownum,review_title,prize_author,bestseller_year,is_bestseller,obscure,firstpub,birthyear,us_national,lowest_date,standard_auth
0,uc1.b3347126,"shaw, irwin","welcome to the city, and other",1942.0,1942.0,3289.0,"welcome to the city, and other stories.",False,NaN,False,False,1942,1913.0,True,1942,"shaw, irwin"
1,uc1.$b322688,"bennett, arnold",woman who stole everything,1927.0,1927.0,4140.0,"woman who stole everything, and other stories.",False,NaN,False,False,1927,1867.0,False,1927,"bennett, arnold"
2,uc1.b3407035,"lippincott, joseph wharton",wilderness champion; the story,1944.0,1944.0,3481.0,wilderness champion; the story of a great houn...,False,NaN,False,False,1944,1887.0,True,1944,"lippincott, joseph w"
3,uc1.$b116192,"becker, may lamberton",golden tales of the southwest;,1939.0,1939.0,4283.0,golden tales of the southwest; selected with a...,False,NaN,False,False,1939,1873.0,True,1939,"becker, may lamberto"
4,mdp.39015059415383,"miller, alice duer, mrs",instruments,1926.0,1926.0,2405.0,"instruments of darkness, and other stories.",False,NaN,False,False,1926,1874.0,True,1926,"miller, alice duer,"


In [26]:
docdict['uc1.$b116192']

(3.70651992663, 930.0)

In [27]:
newsentcol = []
newwordcol = []
for doc in match.docid:
    if doc in docdict:
        sentiment, wordcount = docdict[doc]
    else:
        sentiment = np.nan
        wordcount = 0
    
    newsentcol.append(sentiment)
    newwordcol.append(wordcount)
        

In [28]:
match.shape

(5497, 16)

In [29]:
len(newwordcol)

5497

In [30]:
match['review_wordcount'] = newwordcol

In [31]:
match['review_sentiment'] = newsentcol

In [32]:
match.to_csv('bookswithreception.tsv', sep = '\t', index = False)